In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, get_linear_schedule_with_warmup
import sys
import pandas as pd
import numpy as np
import string
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
import os
import torch.nn.functional as F

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels = 21,output_attentions = False,output_hidden_states = False)
optimizer = torch.optim.AdamW(model.parameters(),lr = 2e-5, eps = 1e-8)
current_directory = "C:\\07-Project\\Final Project 8th Sem\\01-NLP\\"

In [ ]:
data_training = pd.read_csv(os.path.join(current_directory,"Custom-Dataset.csv"), encoding="utf-8")
data_training.head()
questions = []
target_answers = []
answers = []
labels = []
max_len = 0

for index,row in data_training.iterrows():
    target_answers.append(row["reference_answer_en"].translate(str.maketrans('', '', string.punctuation)).lower())
    answers.append(row["student_answer_en"].translate(str.maketrans('', '', string.punctuation)).lower())
    labels.append(row["Average_Mark"])

for i in range(0,len(answers)):
    input_ids = tokenizer.encode(answers[i],target_answers[i], add_special_tokens=True)
    max_len = max(max_len, len(input_ids))

In [ ]:
def preprocessing_for_bert(answers,target_answers):
    input_ids = []
    attention_masks = []
    token_type_ids = []
    for i in range(0,len(answers)):
        encoded_dict = tokenizer.encode_plus(
                        answers[i],
                        target_answers[i],
                        add_special_tokens = True,
                        max_length = max_len,
                        truncation = True,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
        input_ids.append(encoded_dict['input_ids'])
        token_type_ids.append(encoded_dict['token_type_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    return input_ids,token_type_ids, attention_masks

In [ ]:
input_ids,token_type_ids, attention_masks = preprocessing_for_bert(answers,target_answers)
input_ids = torch.cat(input_ids, dim=0)
token_type_ids = torch.cat(token_type_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)
index = data_training.index.tolist()
tensor_index = torch.tensor(index)
dataset = TensorDataset(input_ids, attention_masks,token_type_ids, labels,tensor_index)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_data, val_data = random_split(dataset, [train_size, val_size])

In [ ]:
torch.save(train_data,os.path.join(current_directory, "dataset", "train_dataset.pt"))
torch.save(val_data,os.path.join(current_directory, "dataset", "test_dataset.pt"))

In [ ]:
train_data = torch.load(os.path.join(current_directory, "dataset", "train_dataset.pt"))
val_data = torch.load(os.path.join(current_directory, "dataset", "test_dataset.pt"))

In [ ]:
batch_size = 32
train_dataloader = DataLoader(train_data,sampler = RandomSampler(train_data),batch_size = batch_size)
validation_dataloader = DataLoader(val_data,sampler = SequentialSampler(val_data),batch_size = batch_size)
epochs = 4
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0,num_training_steps = total_steps)

In [ ]:
import numpy as np
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from math import sqrt
def flat_accuracy(preds, labels):
    pred_flat = preds.flatten()
    labels_flat = labels.flatten()
    return pearsonr(pred_flat,labels_flat)[0]
def flat_rmse(preds, labels):
    pred_flat = preds.flatten()
    labels_flat = labels.flatten()
    return sqrt(mean_squared_error(pred_flat,labels_flat))
def weights_init(m):
  torch.nn.init.xavier_uniform(m.weight.data)

In [ ]:
global predictions, indexes, labels_test
predictions = np.array([])
indexes = np.array([])
labels_test = np.array([])
def get_predictions(p,l,i):
  global predictions, indexes, labels_test
  predictions = np.concatenate((predictions,p.flatten()),axis=0)
  indexes = np.concatenate((indexes,i.flatten()),axis=0)
  labels_test = np.concatenate((labels_test,l.flatten()), axis=0)
def flat_accuracy_classification_task(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

## Train

In [ ]:
import random
import numpy as np
import torch
import torch.nn.functional as F
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
training_stats = []
total_t0 = time.time()

for epoch_i in range(0, epochs):
    t0 = time.time()
    total_train_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
        if step % 10 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('Batch:', step, "Time:", elapsed)
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_type_token_ids = batch[2].to(device)
        b_labels = batch[3].to(device)
        b_index = batch[4].to(device)
        model.zero_grad()

        rounded_labels = torch.round(b_labels)
        mapped_labels = rounded_labels.clamp(min=0, max=5)
        one_hot_labels = F.one_hot(mapped_labels.long(), num_classes=6).float()
        outputs = model(b_input_ids, 
                        token_type_ids=b_type_token_ids, 
                        attention_mask=b_input_mask,
                        labels=one_hot_labels)
        
        loss = outputs.loss
        logits = outputs.logits
        total_grades = 0
        l = [0,1,2,3,4,5]
        for logits_batch in logits:
            probabilities = F.softmax(logits_batch)
            batch_grade = sum(probabilities[i] * l[i] for i in range(len(probabilities)))
            print(batch_grade)
            total_grades += batch_grade.item()

        print("Total grades for the batch:", total_grades)
        
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
    avg_train_loss = total_train_loss / len(train_dataloader)            
    training_time = format_time(time.time() - t0)

    print("\nAverage training loss:", avg_train_loss)
    print("Training epoch took:", training_time)
    print("Running Validation...")

    t0 = time.time()
    model.eval()
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_type_token_ids = batch[2].to(device)
        b_labels = batch[3].to(device)
        b_index = batch[4]
        with torch.no_grad():
            output = model(b_input_ids, token_type_ids=b_type_token_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        logits = outputs.logits
        total_eval_loss += loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        get_predictions(logits,label_ids,b_index.numpy())

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    validation_time = format_time(time.time() - t0)
    print("  Validation Loss:", round(avg_val_loss, 2))
    
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("Total Time:", format_time(time.time()-total_t0))


In [ ]:
torch.save(model.state_dict(), os.path.join(current_directory, "BERT_Model","BERT_MultiModel.pth"))